In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 26 15:08:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install transformers

In [ ]:
HF_TOKEN = "YOUR_HF_TOKEN_HERE"  # Replace with your actual Hugging Face token from https://huggingface.co/settings/tokens

In [ ]:
import os

In [ ]:
os.environ["HF_TOKEN"] = HF_TOKEN

YOU NEED TO TELL OS ABOUT THE HUGGING FACE TOKEN

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

THE ABOVE CELL IS NEEDED BECAUSE A MODEL NEEDS A DEVICE

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "google/gemma-3-1b-it"

IT WILL LOAD MODEL AND ITS PARAMETERS (HOW DO WE KNOW THE PARAMETERS? ITS GIVEN IN THE MODEL NAME 1B MEANS 1 BILLION PARAMETERS)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
tokenizer("Hello, World!")

{'input_ids': [2, 9259, 236764, 4109, 236888], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
input_conversation = [
    {"role": "user", "content": "Which is the best place to learn GenAI?"},
    {"role": "assistant", "content":"The best place to learn AI is"}
]

In [ ]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=True,
)
input_tokens

[2,
 105,
 2364,
 107,
 24249,
 563,
 506,
 1791,
 1977,
 531,
 3449,
 8471,
 12553,
 236881,
 106,
 107,
 105,
 4368,
 107,
 818,
 1791,
 1977,
 531,
 3449,
 12498,
 563,
 106,
 107]

GIVES YOU IN TOKENIZED FORMAT OF GEMMA

In [ ]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
)
input_tokens

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is<end_of_turn>\n'

GIVES YOU OUTPUT IN RAW FORMAT TELLS YOU BEGINNING OF STRING AND EVERYTHING

In [ ]:
input_detokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
    continue_final_message=True,
)
input_detokens

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI is'

WORKS LIKE A CHAT TEMPLATE

In [ ]:
output_label = "GenAI Cohort 1.0 by ChaiCode and Piyush Garg"
full_conversation = input_detokens + output_label + tokenizer.eos_token
full_conversation

'<bos><start_of_turn>user\nWhich is the best place to learn GenAI?<end_of_turn>\n<start_of_turn>model\nThe best place to learn AI isGenAI Cohort 1.0 by ChaiCode and Piyush Garg<eos>'

ITS LIKE PREPARING A DATASET FOR THE RESULT YOU WANT IN HERE WE HAVE ONE BUT IN REALITY YOU MIGHT HAVE MANY(BASICALLY YOU ARE MODELLING YOUR DESIRED RESPONSE AND NO ITS NOT SAME AS SYSTEM PROMPT)

In [ ]:
input_tokenized = tokenizer(full_conversation, return_tensors="pt", add_special_tokens=False).to(device)["input_ids"]
input_tokenized

tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107,    105,   4368,
            107,    818,   1791,   1977,    531,   3449,  12498,    563,  14696,
          12553, 105657,    632, 236743, 236770, 236761, 236771,    684, 119806,
           4809,    532, 168222,   1974, 102629,      1]], device='cuda:0')

In [ ]:
input_ids = input_tokenized[:, :-1].to(device)
target_ids = input_tokenized[:, 1:].to(device)
print(f"input_ids: {input_ids}")
print(f"target_ids: {target_ids}")

input_ids: tensor([[     2,    105,   2364,    107,  24249,    563,    506,   1791,   1977,
            531,   3449,   8471,  12553, 236881,    106,    107,    105,   4368,
            107,    818,   1791,   1977,    531,   3449,  12498,    563,  14696,
          12553, 105657,    632, 236743, 236770, 236761, 236771,    684, 119806,
           4809,    532, 168222,   1974, 102629]], device='cuda:0')
target_ids: tensor([[   105,   2364,    107,  24249,    563,    506,   1791,   1977,    531,
           3449,   8471,  12553, 236881,    106,    107,    105,   4368,    107,
            818,   1791,   1977,    531,   3449,  12498,    563,  14696,  12553,
         105657,    632, 236743, 236770, 236761, 236771,    684, 119806,   4809,
            532, 168222,   1974, 102629,      1]], device='cuda:0')


ITS LIKE SLIDING WINDOW METHOD WE ARE TELLING THE MODEL THAT IF THE INPUT IS 12345 THEN THE OUTPUT IS MINUS THE 1 AND THE REMAINING TOKENS IS YOUR OUTPUT LIKE 2345*8* , 8 OVER HERE IS THE NEW  TARGETED OUTPUT
WE DO THIS BECAUSE ITS NECESSARY TO DIFFERENTIATE INPUT AND OUTPUT SO WE CAN CALCULATE THE LOSS FOR FINE TUNING

In [ ]:
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fn = nn.CrossEntropyLoss(reduction="none")
    cross_entropy = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
    return cross_entropy

CALCULATE LOSS FUNCTION

In [ ]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
).to(device)

MODEL PULL

In [ ]:
from torch.optim import AdamW
model.train()

optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

for _ in range(10):
  out = model(input_ids=input_ids)
  loss = calculate_loss(out.logits, target_ids).mean()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(loss.item())

11.3125
4.8125
2.6875
1.4609375
0.8359375
0.609375
0.50390625
0.478515625
0.44140625
0.412109375


IN LAYMAN TERMS WHAT IS HAPPENING OVER HERE IS WE ARE FIRST OPTIMIZING THE MODEL THEN WE ARE TELLING THE MODEL TO FETCH THE INPUT WAIT FOR IT TO PROCESS THE OUTPUT ONCE ITS PROCESSED SEE OUT DESIRED OUTPUT AND CALCULATE THE LOSS THEN GO BACK TO THE MODEL AGAIN UPDATE THE BITS AND DO THIS 10 TIME SINCE OUR LOOP RUNS 10 TIMES THEREFORE WE DO THIS IN LOOP FOR MULTIPLE TIMES IS BECAUSE WE KEEP CALCULATING THE LOSS UPDATING THE BITS TILL WE GET THE DESIRED OUTPUT WHICH IS CALLED FINE TUNING

In [ ]:
input_prompt = [
    { "role": "user", "content": "Which is the best place to learn GenAI?" }
]

input = tokenizer.apply_chat_template(
    conversation=input_prompt,
    return_tensors="pt",
    tokenize=True,
).to(device)

output = model.generate(input, max_new_tokens=35)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

['user\nWhich is the best place to learn GenAI?\nmodel\nThe best place to learn AI isGenAI Cohort 1.0 by ChaiCode and Piyush Garg']


In [ ]:
THIS IS WHAT WE ARE GETTING NOW THE FINETUNED OUTPUT

In [ ]:
input_prompt = "Which is the best place to learn GenAI?"
i_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to(device)

output_tokens = model.generate(i_tokens)
tokenizer.batch_decode(output_tokens)

['<bos>Which is the best place to learn GenAI?\n\nThere\'s no single "best" place, but here\'s a breakdown of excellent resources']

AND THIS IS WHAT WE WERE GETTING BEFORE WE FINETUNED